In [1]:
import pandas as pd

In [2]:
from datetime import datetime, timedelta

## Ingest export from ServiceNow

In [3]:
src_file = input()

 src_data/u_bkop_incident_230603.csv


In [4]:
df = pd.read_csv(
    src_file,
    parse_dates=["created", "resolved"],
    low_memory=False
)

In [5]:
df.shape[0]

121479

In [6]:
assert df.columns.tolist() == [
    "number", "created", "caller", "state", "category", "subcategory",
    "assignment_group", "bpl_location", "nyp_location", "resolved",
    "assigned_to", "system", "mat_source", "close_code", "url",
    "reassignment_count", "updates", "resolved_by"]

In [7]:
df["category"].unique()

array(['Cataloging', 'Acquisitions', 'Selection', 'Logistics',
       'Offsite - ReCAP', 'Collections Processing', nan], dtype=object)

### Narrow down to Cataloging tickets: orginated as CAT or assigned to CAT

In [8]:
cdf = df[(df["category"] == "Cataloging")|(df["assignment_group"] == "BKOPS CAT")]

In [9]:
cdf.shape[0]

21250

## CAT tickets volumne by year

In [13]:
gyear_cdf = cdf.groupby(cdf["created"].map(lambda x: x.year))

In [14]:
ycdf = pd.DataFrame(columns=["year", "tickets"])

In [15]:
for year, d in gyear_cdf:
    new_row = pd.Series({"year": year, "tickets": d.shape[0]})
    ycdf = pd.concat([ycdf, new_row.to_frame().T], ignore_index=True)

In [16]:
ycdf

,year,tickets
0,2013,1311
1,2014,2140
2,2015,2739
3,2016,2554
4,2017,2233
5,2018,2112
6,2019,1785
7,2020,955
8,2021,1964
9,2022,2169


In [17]:
ycdf.to_csv("public_data/cat-by-year.csv", index=False)

### Expected volume in the most recent year

In [18]:
## assume the last row in cdf shows cutoff date for the analysis

In [19]:
last_day = cdf.iloc[-1]["created"]

In [20]:
days_to_date = (last_day - datetime(last_day.year, 1, 1)).days  # warning, there must be enough to date data for such prediction to make sense

In [21]:
days_to_go = (datetime(last_day.year, 12, 31) - last_day).days

In [22]:
# calculate expected volume proportionally

In [23]:
tickets_to_date = ycdf.iloc[-1]["tickets"]

In [24]:
expected_current_year_volume = round((days_to_go * tickets_to_date) / days_to_date) + tickets_to_date

In [25]:
print(f"Expected volume in {last_day.year}: {expected_current_year_volume}")

Expected volume in 2023: 3056


### Volume breakdown by system

In [26]:
ycdf_rl = pd.DataFrame(columns=["system", "year", "tickets"])
ycdf_bl = pd.DataFrame(columns=["system", "year", "tickets"])
ycdf_bp = pd.DataFrame(columns=["system", "year", "tickets"])

In [27]:
for year, d in gyear_cdf:
    rl_df = d[d["system"] == "NYPL Research"]
    bl_df = d[d["system"] == "NYPL Circulating"]
    bp_df = d[d["system"] == "BPL Circulating"]
    new_rl = pd.Series({"system": "NYPL RL", "year": year, "tickets" : rl_df.shape[0]})
    new_bl = pd.Series({"system": "NYPL BL", "year": year, "tickets": bl_df.shape[0]})
    new_bp = pd.Series({"system": "BPL", "year": year, "tickets": bp_df.shape[0]})
    ycdf_rl = pd.concat([ycdf_rl, new_rl.to_frame().T], ignore_index=True)
    ycdf_bl = pd.concat([ycdf_bl, new_bl.to_frame().T], ignore_index=True)
    ycdf_bp = pd.concat([ycdf_bp, new_bp.to_frame().T], ignore_index=True)

In [28]:
ycdf_rl.to_csv("public_data/rl-by-year.csv", index=False)
ycdf_bl.to_csv("public_data/bl-by-year.csv", index=False)
ycdf_bp.to_csv("public_data/bp-by-year.csv", index=False)

#### Expected volume by system

In [29]:
rl_tickets_to_date = ycdf_rl.iloc[-1]["tickets"]

In [30]:
bl_tickets_to_date = ycdf_bl.iloc[-1]["tickets"]

In [31]:
bp_tickets_to_date = ycdf_bp.iloc[-1]["tickets"]

In [32]:
expected_current_year_volume_rl = round((days_to_go * rl_tickets_to_date) / days_to_date) + rl_tickets_to_date

In [33]:
expected_current_year_volume_bl = round((days_to_go * bl_tickets_to_date) / days_to_date) + bl_tickets_to_date

In [34]:
expected_current_year_volume_bp = round((days_to_go * bp_tickets_to_date) / days_to_date) + bp_tickets_to_date

In [35]:
print(f"Expected NYPL RL volume in {last_day.year}: {expected_current_year_volume_rl}")
print(f"Expected NYPL BL volume in {last_day.year}: {expected_current_year_volume_bl}")
print(f"Expected NYPL BPL volume in {last_day.year}: {expected_current_year_volume_bp}")

Expected NYPL RL volume in 2023: 235
Expected NYPL BL volume in 2023: 1599
Expected NYPL BPL volume in 2023: 1222


## Last year most frequent requesting locations

In [36]:
# treat three systems separately

In [37]:
start_date_str = input()  # enter format: YYYY-MM-DD

 2023-06-04


In [38]:
start_date = datetime.strptime(start_date_str, "%Y-%m-%d") - timedelta(365)
print(start_date)

2022-06-04 00:00:00


#### NYPL Research

In [40]:
loc_rl = cdf[(cdf["system"] == "NYPL Research") & (cdf["created"] >= start_date)]

In [41]:
loc_rl.shape[0]

251

In [42]:
gloc_rl = loc_rl["nyp_location"].value_counts()

In [43]:
gloc_rl.to_csv("public_data/freq-loc-rl.csv")

In [44]:
caller_rl = loc_rl["caller"].value_counts()

In [45]:
caller_rl.to_csv("private_data/callers-rl.csv")

#### NYPL Circ

In [46]:
loc_bl = cdf[(cdf["system"] == "NYPL Circulating") & (cdf["created"] >= start_date)]

In [47]:
loc_bl.shape[0]

1342

In [48]:
gloc_bl = loc_bl["nyp_location"].value_counts()

In [49]:
gloc_bl.head()

nyp_location
Bronx Library Center                   140
Stavros Niarchos Foundation Library    113
Library for the Performing Arts         93
Stephen A. Schwarzman Building          69
Seward Park Library                     61
Name: count, dtype: int64

In [50]:
gloc_bl.tail()

nyp_location
Bookmobile, Manhattan     1
Huguenot Park Library     1
270MAD                    1
Clason's Point Library    1
Eastchester Library       1
Name: count, dtype: int64

In [51]:
gloc_bl.to_csv("public_data/freq-loc-bl.csv")

In [52]:
caller_bl = loc_bl["caller"].value_counts()

In [53]:
caller_bl.to_csv("private_data/callers-bl.csv")

#### BPL

In [54]:
loc_bp = cdf[(cdf["system"] == "BPL Circulating") & (cdf["created"] >= start_date)]

In [55]:
loc_bp.shape[0]

1035

In [56]:
gloc_bp = loc_bp["bpl_location"].value_counts()

In [57]:
gloc_bp.head()

bpl_location
Central Literature & Languages - 14    149
Brooklyn Heights - 50                  128
Central SST - 16                        96
Windsor Terrace - 77                    55
Central HBR (Hist/Biog/Rel) - 13        54
Name: count, dtype: int64

In [58]:
gloc_bp.tail()

bpl_location
Central - Mailroom                  1
Center for Brooklyn History - 91    1
Brownsville - 27                    1
Marcy - 59                          1
Central Juv Children's Room - 2     1
Name: count, dtype: int64

In [59]:
gloc_bp.to_csv("public_data/freq-loc-bp.csv")

In [60]:
caller_bp = loc_bp["caller"].value_counts()

In [61]:
caller_bp.to_csv("private_data/callers-bp.csv")

#### Consider for the end-of-the-year analysis to include inactive locations (0-5 tickets per year)

## Ticket Categories

In [62]:
# consider last 365 days only!

In [63]:
cat_rl = loc_rl["subcategory"].value_counts()

In [64]:
cat_rl.to_csv("public_data/cat-pie-rl.csv")

In [65]:
cat_bl = loc_bl["subcategory"].value_counts()

In [66]:
cat_bl.to_csv("public_data/cat-pie-bl.csv")

In [67]:
cat_bp = loc_bp["subcategory"].value_counts()

In [68]:
cat_bp.to_csv("public_data/cat-pie-bp.csv")

### Average agent workload

In [69]:
# must split between systems
# RL - 5, BL - 6, BP - 6

In [70]:
ave_rl_tickets_per_agent = (loc_rl.shape[0] / 52.1) / 5

In [71]:
print(f"Avg. NYPL RL agent workload: {ave_rl_tickets_per_agent} in the last 365 days.")

Avg. NYPL RL agent workload: 0.963531669865643 in the last 365 days.


In [72]:
ave_bl_tickets_per_agent_per_week = (loc_bl.shape[0] / 52.1) / 6

In [73]:
print(f"Avg. NYPL BL agent workload: {ave_bl_tickets_per_agent_per_week} in the last 365 days.")

Avg. NYPL BL agent workload: 4.293026231605886 in the last 365 days.


In [74]:
ave_bp_tickets_per_agent_per_week = (loc_bp.shape[0] / 52.1) / 6

In [75]:
print(f"Avg. BPL agent workload: {ave_bp_tickets_per_agent_per_week} in the last 365 days.")

Avg. BPL agent workload: 3.310940499040307 in the last 365 days.


In [76]:
ave_circ_tickets_per_agent_per_week = ((loc_bl.shape[0] + loc_bp.shape[0]) / 52.1) / 12

In [77]:
print(f"Avg. NYPL BL + BPL agent workload: {ave_circ_tickets_per_agent_per_week} in the last 365 days.")

Avg. NYPL BL + BPL agent workload: 3.8019833653230966 in the last 365 days.


In [78]:
# before and after Jan 19, 2023

In [79]:
point_date = datetime(2023, 1, 19)
start_date = datetime(2023, 1, 19) - timedelta(365)

In [80]:
print(start_date, point_date)

2022-01-19 00:00:00 2023-01-19 00:00:00


In [81]:
before_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["resolved"] >= start_date) & (cdf["resolved"] < point_date)]

In [82]:
before_df.shape[0]

1859

In [83]:
after_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["resolved"] >= point_date)]

In [84]:
after_df.shape[0]

1825

In [85]:
agents_before_weekly = pd.DataFrame(columns=["agent", "tickets"]).iloc[0:0]
agents_before = pd.DataFrame(columns=["agent", "tickets"]).iloc[0:0]

In [86]:
for agent, d in before_df.groupby("resolved_by"):
    new_row_weekly = pd.Series({"agent": agent, "tickets": d.shape[0] / 52.1})
    new_row = pd.Series({"agent": agent, "tickets": d.shape[0]})
    agents_before_weekly = pd.concat([agents_before_weekly, new_row_weekly.to_frame().T], ignore_index=True)
    agents_before = pd.concat([agents_before, new_row.to_frame().T], ignore_index=True)

In [87]:
agents_before = agents_before.sort_values("tickets", ascending=False)
agents_before_weekly = agents_before_weekly.sort_values("tickets", ascending=False)

In [88]:
agents_before.to_csv("private_data/agents-circ-before.csv", index=False)
agents_before_weekly.to_csv("private_data/agents-circ-before-weekly.csv", index=False)

In [89]:
agents_after = pd.DataFrame(columns=["agent", "tickets"]).iloc[0:0]
agents_after_weekly = pd.DataFrame(columns=["agent", "tickets"]).iloc[0:0]

In [90]:
for agent, d in after_df.groupby("resolved_by"):
    new_row_weekly = pd.Series({"agent": agent, "tickets": d.shape[0] / 52.1})
    new_row = pd.Series({"agent": agent, "tickets": d.shape[0]})
    agents_after = pd.concat([agents_after, new_row.to_frame().T], ignore_index=True)
    agents_after_weekly = pd.concat([agents_after_weekly, new_row_weekly.to_frame().T], ignore_index=True)

In [91]:
agents_after = agents_after.sort_values("tickets", ascending=False)
agents_after_weekly = agents_after_weekly.sort_values("tickets", ascending=False)

In [92]:
agents_after.to_csv("private_data/agents-circ-after.csv", index=False)

In [93]:
agents_after_weekly.to_csv("private_data/agents-circ-after-weekly.csv", index=False)

### Circ agents response times before and after (equal periods)

In [33]:
relevant_days = (datetime(2023, 6, 4) - datetime(2023, 1, 1)).days

In [34]:
reorg_date = datetime(2023, 1, 19)

In [35]:
before_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["created"] < datetime(2023, 1, 19)) & (cdf["created"] >= reorg_date - timedelta(relevant_days)) & (cdf["resolved"].notnull())]

In [36]:
before_df.shape[0]

827

In [37]:
before_df["resolved"].unique()

<DatetimeArray>
['2022-08-18 12:05:05', '2022-08-18 12:48:18', '2022-08-19 07:35:52',
 '2022-08-18 15:10:34', '2022-08-18 16:19:13', '2022-08-19 10:30:15',
 '2022-08-19 10:27:41', '2022-08-19 14:28:02', '2022-08-22 14:15:29',
 '2022-08-22 13:39:02',
 ...
 '2023-01-19 09:47:53', '2023-01-23 09:17:10', '2023-01-20 16:08:51',
 '2023-01-17 15:59:04', '2023-01-26 14:12:56', '2023-01-18 15:24:40',
 '2023-01-23 10:28:14', '2023-01-31 08:54:31', '2023-01-19 09:46:00',
 '2023-01-27 12:18:09']
Length: 826, dtype: datetime64[ns]

In [38]:
after_df = cdf[(cdf["system"] != "NYPL Research") & (cdf["created"] >= datetime(2023, 1, 19)) & (cdf["resolved"].notnull())]

In [39]:
after_df.shape[0]

1045

In [40]:
after_df["resolved"].unique()

<DatetimeArray>
['2023-01-19 16:52:31', '2023-01-19 13:34:33', '2023-01-23 15:35:38',
 '2023-01-23 15:41:01', '2023-01-20 11:18:22', '2023-01-20 11:04:47',
 '2023-01-20 11:17:03', '2023-01-20 10:32:29', '2023-01-20 10:18:42',
 '2023-01-23 09:00:14',
 ...
 '2023-06-01 10:10:36', '2023-06-01 09:25:47', '2023-06-01 11:52:41',
 '2023-06-02 12:18:13', '2023-06-01 15:22:57', '2023-06-02 09:56:36',
 '2023-06-02 09:58:57', '2023-06-02 10:32:44', '2023-06-02 14:52:23',
 '2023-06-02 12:54:16']
Length: 1045, dtype: datetime64[ns]

In [45]:
before_df["time2resolve"] = (before_df["resolved"] - before_df["created"]).dt.days

C:\Users\Tomek\AppData\Local\Temp\ipykernel_11128\2241171427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_df["time2resolve"] = (before_df["resolved"] - before_df["created"]).dt.days


In [46]:
before_df["period"] = "before"

C:\Users\Tomek\AppData\Local\Temp\ipykernel_11128\1603314720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_df["period"] = "before"


In [47]:
before_df["time2resolve"].describe()

count    827.000000
mean      32.821040
std       50.508738
min        0.000000
25%        0.000000
50%        9.000000
75%       43.500000
max      265.000000
Name: time2resolve, dtype: float64

In [48]:
before_df.to_csv("public_data/time2resolve-circ-before-all-points.csv", columns=["number", "time2resolve", "period"], index=False)

In [49]:
after_df["time2resolve"] = (after_df["resolved"] - after_df["created"]).dt.days

C:\Users\Tomek\AppData\Local\Temp\ipykernel_11128\167934099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_df["time2resolve"] = (after_df["resolved"] - after_df["created"]).dt.days


In [50]:
after_df["period"] = "after"

C:\Users\Tomek\AppData\Local\Temp\ipykernel_11128\3740744792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_df["period"] = "after"


In [51]:
after_df["time2resolve"].describe()

count    1045.000000
mean        2.440191
std         9.257880
min         0.000000
25%         0.000000
50%         0.000000
75%         2.000000
max       124.000000
Name: time2resolve, dtype: float64

In [52]:
after_df.to_csv("public_data/time2resolve-circ-after-all-points.csv", columns=["number", "time2resolve"], index=False)

In [53]:
time2resolve_df = pd.concat([before_df, after_df], ignore_index=True)

In [54]:
time2resolve_df.shape[0]

1872

In [55]:
time2resolve_df.to_csv("public_data/time2resolve-circ-all-points.csv", columns=["number", "time2resolve", "period"], index=False)